In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
print(tf.__version__)

print(tf.config.experimental.list_physical_devices('GPU')[0])

2.3.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
!nvidia-smi

Sun Aug 16 10:16:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |    229MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 6.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7437a40e7b16037571346bdd291453ff95a06cc0375b4d8167de724ab5ab0bc4
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=9b2214727ace10b7000eed402162f2eb1bd02fcd613c7ad195d0224a0a2c2e2b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import os,logging,sys,random
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger,TensorBoard
from tensorboard.plugins.hparams import api as hp
import kerastuner as kt

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

logging.disable(logging.WARNING)

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Load the fashion-mnist pre-shuffled train data and test data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

x_train shape: (60000, 28, 28) y_train shape: (60000,)


In [ ]:
x_train=np.expand_dims(x_train,axis=-1)
x_test=np.expand_dims(x_test,axis=-1)

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
train_x,val_x,train_y,val_y=train_test_split(x_train,y_train,test_size=0.3,random_state=5)

In [ ]:
train_y_one_hot=to_categorical(train_y)
val_y_one_hot=to_categorical(val_y)
y_test_one_hot=to_categorical(y_test)

In [ ]:

def build_model(hp):
  inputs = tf.keras.Input(shape=(32, 32, 3))
  x = inputs
  input_size=(*TRAIN_DIM, N_CHANNELS)
  inputs = tf.keras.Input(shape=input_size)
  x=Flatten(input_shape=(28,28))(inputs)
  
  dense_1=Dense(hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
  
  drop_out=Dropout(hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(dense_1)

  
  outputs = Dense(10, activation=tf.nn.softmax)(drop_out)
  
  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='categorical_crossentropy', 
    metrics=['accuracy'])
  return model

In [ ]:
import kerastuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2)

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


In [ ]:
tuner.search(train_x, train_y_one_hot,
             validation_data=(val_x,val_y_one_hot),
             epochs=2,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1),ClearTrainingOutput])

Epoch 1/2
1313/1313 [==============================] - 4s 3ms/step - loss: 1.0472 - accuracy: 0.6457 - val_loss: 0.6651 - val_accuracy: 0.7776
Epoch 2/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6838 - accuracy: 0.7723 - val_loss: 0.5561 - val_accuracy: 0.8179


Epoch 1/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.9239 - accuracy: 0.6885 - val_loss: 0.5974 - val_accuracy: 0.7949
Epoch 2/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6134 - accuracy: 0.7922 - val_loss: 0.5171 - val_accuracy: 0.8204


Epoch 1/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6393 - accuracy: 0.7714 - val_loss: 0.4443 - val_accuracy: 0.8379
Epoch 2/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.5002 - accuracy: 0.8163 - val_loss: 0.4660 - val_accuracy: 0.8179


Epoch 1/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.7752 - accuracy: 0.7211 - val_loss: 0.4855 - val_accuracy: 0.8254
Epoch 2/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.5803 - accuracy: 0.7915 - val_loss: 0.4498 - val_accuracy: 0.8298


Epoch 1/2
1313/1313 [==============================] - 4s 3ms/step - loss: 1.3655 - accuracy: 0.5393 - val_loss: 0.8174 - val_accuracy: 0.7540
Epoch 2/2
1313/1313 [==============================] - 4s 3ms/step - loss: 0.9180 - accuracy: 0.6931 - val_loss: 0.6530 - val_accuracy: 0.7999


...

Epoch 1/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.6505 - accuracy: 0.7730 - val_loss: 0.4566 - val_accuracy: 0.8350
Epoch 2/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4736 - accuracy: 0.8306 - val_loss: 0.4216 - val_accuracy: 0.8446
Epoch 3/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4415 - accuracy: 0.8418 - val_loss: 0.4113 - val_accuracy: 0.8540
Epoch 4/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4178 - accuracy: 0.8497 - val_loss: 0.3822 - val_accuracy: 0.8613
Epoch 5/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4073 - accuracy: 0.8512 - val_loss: 0.3742 - val_accuracy: 0.8638
Epoch 6/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.3948 - accuracy: 0.8557 - val_loss: 0.3970 - val_accuracy: 0.8506


Epoch 1/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.5282 - accuracy: 0.8130 - val_loss: 0.4334 - val_accuracy: 0.8439
Epoch 2/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4011 - accuracy: 0.8553 - val_loss: 0.4594 - val_accuracy: 0.8254


Epoch 1/10
1313/1313 [==============================] - 5s 3ms/step - loss: 0.5913 - accuracy: 0.7864 - val_loss: 0.4327 - val_accuracy: 0.8409
Epoch 2/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4825 - accuracy: 0.8235 - val_loss: 0.5101 - val_accuracy: 0.8077


Epoch 1/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.7038 - accuracy: 0.7511 - val_loss: 0.4698 - val_accuracy: 0.8322
Epoch 2/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.5079 - accuracy: 0.8211 - val_loss: 0.4330 - val_accuracy: 0.8412
Epoch 3/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4666 - accuracy: 0.8337 - val_loss: 0.4078 - val_accuracy: 0.8526
Epoch 4/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4424 - accuracy: 0.8417 - val_loss: 0.3825 - val_accuracy: 0.8627
Epoch 5/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4253 - accuracy: 0.8479 - val_loss: 0.3747 - val_accuracy: 0.8626
Epoch 6/10
1313/1313 [==============================] - 4s 3ms/step - loss: 0.4135 - accuracy: 0.8499 - val_loss: 0.3752 - val_accuracy: 0.8623


Epoch 1/10
1300/1313 [============================>.] - ETA: 0s - loss: 0.6898 - accuracy: 0.7674

In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [ ]:
test_loss, test_acc = model.evaluate(x_test,y_test_one_hot)

313/313 [==============================] - 1s 3ms/step - loss: 0.3872 - accuracy: 0.8774


In [ ]:
best_hyperparameters

In [ ]:
test_pred = best_model.predict(x_test)

In [ ]:
print("Test Accuracy:",test_acc)

Test Accuracy: 0.8773999810218811


In [ ]:
out=np.argmax(test_pred,axis=-1)

In [ ]:
confusion_matrix(y_test,out)

array([[723,   1,  15,  67,   5,   1, 182,   0,   6,   0],
       [  4, 969,   2,  18,   4,   0,   3,   0,   0,   0],
       [  7,   0, 831,  17,  75,   0,  69,   0,   1,   0],
       [  7,   4,  10, 920,  30,   1,  23,   0,   5,   0],
       [  0,   0, 113,  35, 795,   0,  54,   0,   3,   0],
       [  0,   0,   0,   1,   0, 968,   0,  20,   1,  10],
       [ 70,   0,  99,  45,  60,   0, 721,   0,   5,   0],
       [  0,   0,   0,   0,   0,  19,   0, 969,   1,  11],
       [  2,   0,   3,   9,   3,   5,   3,   3, 972,   0],
       [  0,   0,   0,   0,   0,  12,   0,  82,   0, 906]])